In the third article of the series, Using Machine Learning to Predict the Weather: Part 3, I describe how the processes and steps required to build a Neural Network using Google's TensorFlow to predict future mean daily temperatures. Using this method I can then compare the results to the Linear Regression model.

Topics I will be covering in this article include:

- Understanding Artificial Neural Networks Theory
- TensorFlow's High Level Estimator API
- Building a DNNRegressor to Predict the Weather

## Understanding Artificial Neural Networks Theory

#### En el notebook anterior he utilizado la tecnica de regresion lineal, la cual es muy util, pero rpesenta el problema de que se supone que hay una relacion lineal entre las variables a estudiar.

#### En este caso voy a utilizar redes neuronales, que se comportan bastante bien tanto cuando hay relaciones lineales como cuando no.

#### Se trata de un aprendizaje supervisado, ya el modelo se va a entrenar con informacion que ya se sabe su salida.

TODO --> poner mas informacion sobre como funciona una red neuronal

## TensorFlow's High Level Estimator API